# Handrít.is – Datenexport

Dieses Notebook enthält ein Script, das einen **Datenexport** von **handrit.is** ermöglicht.\
Handrít ist eine Datenbank, die fast sämtliche isländische/westnordische Handschriften enthält.\
Nützlich für die Forschung von GeisteswissenschaftlerInnen ist insbesondere ein Schlagwortsystem, das das Auffinden von Manuskripten ermöglicht.

## Technische Voraussetzungen
* Diese Datei ist ein [*Jupyter Notebook*](https://de.m.wikipedia.org/wiki/Project_Jupyter#Jupyter_Notebook) und verwendet die Programmiersprache [*Python*](https://de.m.wikipedia.org/wiki/Python_(Programmiersprache)).
* Um das Notebook **lokal** zu verwenden, musst du die Dateien herunterladen und sowohl Python als auch Jupyter-Notebook auf deinem Rechner installieren. Es gibt im Web massenhaft Tutorials zur Installation der beiden Komponenten. Python kann direkt von der offiziellen [Website](https://www.python.org/downloads/) der *Python Software Foundation* heruntergeladen und als .exe installiert werden (bei der Installation sollte unbedingt die Option gewählt werden, Python zum **PATH** hinzuzufügen!). Das Projekt Jupyter bietet ein ausführliches [Installations-Tutorial](https://jupyter.org/install) auf seiner Website.
* Möchtest du Python & Jupyter nicht installieren, kannst du das Notebook stattdessen in einer Browserapplikation laden. Hierzu bietet sich z. B. [Google Colaboratory](https://colab.research.google.com/) an.
* Falls du dich vertieft mit Python auseinandersetzen möchtest, existieren im Web viele gute (und leider auch unfassbar viele schlechte) Ressourcen. Ich persönlich kann die Videos des Hamburger Mathematikers [Edmund Weitz](https://www.youtube.com/@WeitzHAWHamburg/featured) sowie des Informatik-Professors [Jens Dittrich](https://www.youtube.com/@jensdit) von der Uni Saarland empfehlen. Beide bieten sowohl Zugang zu den absoluten Grundlagen als auch zu fortgeschritteneren und interessanteren Aspekten des Programmierens.

## Grundlagen
* Die Keywords (auf Englisch) sind unter [https://handrit.is/manuscript/browse/keyword?lang=en](https://handrit.is/manuscript/browse/keyword?lang=en) durchsuchbar.
* **Wichtig:** Öffnet man eine Seite, muss man sich alle Manuskripte anzeigen lassen, indem man "Show all" klickt: ![show all](show_all_handr!t.png)

## Variablen befüllen
* Im nächsten Schritt musst du zwei Variablen für das Script definieren:
* **Url** ist die Website des Verzeichnisses, das du exportieren möchtest. **Wichtig:** Du solltest "show all" geklickt haben, sodass am Ende deiner URL die Anzahl der Einträge steht (z. B.: https://handrit.is/manuscript/list/keyword/kon?**size=289**) – **zudem:** die URL muss in Anführungszeichen ("") stehen, sodass Python es als String-Datentyp erkennt!
* **output_name** ist der Dateiname der Textdatei, die als Ergebnis im selben Ordner erstellt wird, wo du auch das Notebook gespeichert hast. By default heißt die .txt-Datei "handrit_output.txt". Du kannst sie ändern, wichtig ist, dass der Name in Anführungszeichen steht!

## Hier definieren:

In [1]:
Url = "https://handrit.is/manuscript/list/keyword/gudfr?size=601"
output_name = "handrit_output_theology.txt"

## Script starten
* Starte nun das Script, indem du oben in der Kontrollleiste auf **"Run" > "Run all cells"** klickst. Alternativ kannst du einzelne Zellen über "Shift" + "Enter" individuell ausführen.* Nach ein paar Sekunden kannst du unten das Ergebnis sehen.
* Den auszuführenden Programmcode kannst du in der nächsten Zelle sehen.

In [12]:
# Alle nächsten Schritte und Variablen dürfen nicht geändert werden, da sie das Programm steuern!
## Import der verwendeten Module:
import requests
from bs4 import BeautifulSoup as BS

## Listen für die Ergebnisse:
safn_list = []
second_list = []
title_list = []
loc_list = []
date_list = []
temp_list = []


## Variablen um die Funktionen zu befüllen:
r = requests.get(Url)
soup = BS(r.text)


## Die einzelnen Kolumnen des Verzeichnisses durch die Funktion "find_all" des Moduls "Beautifulsoup" in Variablen speichern:
safn = soup.find_all('div', class_='col-6 col-lg-2')
second = soup.find_all('div', class_='col-12 col-lg-8')
for i in safn:
    entry = i.get_text()
    safn_list.append(entry)
del safn_list[0]
for i in second:
    second_list.append(i.get_text(separator="|"))
for i in second_list:
    temp_list = i.split(sep="|")
    title_list.append(temp_list[0])
    if "; " in temp_list:
        temp_list.remove("; ")
    if ", " in temp_list:
        temp_list.remove(", ")
    if len(temp_list) == 3:
        loc_list.append(temp_list[1])
        date_list.append(temp_list[2])
    if len(temp_list) == 1:
        loc_list.append("empty")
        date_list.append("empty")
    if len(temp_list) == 2:
        if temp_list[1][-1].isnumeric() == True:
            date_list.append(temp_list[1])
            loc_list.append("empty")
        else:
            loc_list.append(temp_list[1])
            date_list.append("empty")


term_post = []
terminus = []
for i in date_list:
    if i == "empty":
        term_post.append(i)
        terminus.append(i)
    elif "-" not in i:
        term_post.append("n/a")
        terminus.append(i)
    else:
        term_post.append(i[0:i.find("-")])
        terminus.append(i[(i.find("-")+1):])


## Ergebnis:
In der übernächsten Zelle findest du dein Resultat: Jede Zeile ist eine Handschrift deiner Liste. Die Einträge sind mit Semikolon ";" getrennt, sodass du sie als CSV in Excel oder Sheets einfügen kannst. Gleichzeitig wird eine Textfile auf deine Festplatte geschrieben, die den Output enthält. Du solltest sie in deinem "User" Ordner als "handrit_output.txt" finden. Benutze alternativ die Suchfunktion deines Rechners. Viel Erfolg bei deiner Forschung!

In [38]:
file = open(output_name, "w")
file.write("Sigle;Name;Ort;Terminus post;Terminus ante (oder gesichert)\n")
file.close()
for i in range(len(title_list)):
    line = ";".join([safn_list[i],title_list[i],loc_list[i],term_post[i],terminus[i]])+"\n"

    with open(output_name, "a") as output:
        output.write(line)
out = open(output_name, "r")
print(out.read())
out.close()

Sigle;Name;Ort;Ante;Terminus
AM 98 fol.;Historia de antiquitate Regum Norwagiensium;Denmark or Norway;1640;1660
AM 238 XVIII fol.;The Icelandic Elucidarius and a Sermon;Iceland;1500;1550
AM 238 XXVI fol.;Liturgical Texts;Iceland;1390;1410
AM 266 fol.;Bréfabók Gissurs biskups Einarssonar;Ísland;1600;1700
AM 21 4to;Old Danish Legal Manuscript;Denmark;1475;1499
AM 135 4to;Lögbók;empty;empty;empty
AM 281 4to;Miscellany;Iceland;1685;1699
AM 544 4to;Hauksbók;Iceland and Norway;1290;1360
AM 624 4to;Samtíningur;Ísland;1490;1510
AM 626 4to;Theological Manual;empty;empty;empty
AM 655 XXVII 4to;A Collection of Homilies;Iceland;1200;1299
AM 671 4to;Theological and Canonical Legal Manuscript;Iceland;1300;1340
AM 672 4to;Miscellany of Religious Literature;Iceland;1450;1499
AM 673 a III 4to;Teiknibókin;Ísland;1450;1475
AM 674 a 4to;Bókin dýra;Iceland;1150;1199
AM 674 b 4to;Elucidarium;empty;1400;1499
AM 674 c 4to;Elucidarium;empty;1290;1310
AM 675 4to;Hauksbók: Elucidarius;Iceland;1302;1310
AM 684 4t